In [2]:
import datetime
import os
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy

In [3]:
SRC_PATH = os.environ.get('SRC_PATH')  #  path to source code os.path.abspath('.')
DB_NAME = os.environ.get('DB_NAME')  #  [database will be one directory above source code]
IEX_KEY = os.environ.get('IEX_KEY')  # IEX api key to get market data

# Create engine
engine = sqlalchemy.create_engine(f"sqlite:///{SRC_PATH.replace('/MarketView', '')}/{DB_NAME}")

In [3]:
"""

### Uncomment to LOAD sample data into database for testing

pd.read_csv('../sample_data/last_dev.csv').set_index('symbol').to_sql(
    (datetime.datetime.now() - datetime.timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S last'),
    engine) # store development data in database

pd.read_csv('../sample_data/quote_dev.csv').set_index('symbol').to_sql(
    (datetime.datetime.now() - datetime.timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S quote'),
    engine) # store development data in database

"""

In [4]:
# last tables in reverse sorted order
last_feed = sorted([table for table in engine.table_names() if 'last' in table], reverse=True)

last_feed

/home/ec2-user/lab/lab_venv/lib64/python3.7/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['2022-08-02 02:29:53 last',
 '2022-08-02 02:29:01 last',
 '2022-08-02 01:00:02 last',
 '2022-08-02 00:57:09 last',
 '2022-08-02 00:39:43 last',
 '2022-08-02 00:30:32 last']

In [5]:
quote_feed = sorted([table for table in engine.table_names() if 'quote' in table], reverse=True)

quote_feed

/home/ec2-user/lab/lab_venv/lib64/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['2022-08-02 02:29:53 quote',
 '2022-08-02 02:29:02 quote',
 '2022-08-02 01:00:02 quote',
 '2022-08-02 00:57:10 quote',
 '2022-08-02 00:53:52 quote',
 '2022-08-02 00:53:37 quote']

In [6]:
clean_df = \
(
    pd.read_sql(last_feed[0], engine, index_col='symbol').merge(
        pd.read_sql(quote_feed[0], engine, index_col='symbol'),
        on='symbol'
    )
    
    # prepare dataframe
    .assign(
        prev_close = lambda df: df['price'],
        last = lambda df: df['lastSalePrice'],
    )
    
    .drop(columns=[
            'price',
            'bidSize',
            'askSize',
            'size',
            'time',
            'sector',
            'securityType',
            'lastUpdated',
            'lastSalePrice',
            'lastSaleSize',
            'lastSaleTime',
            'volume',
            ]
        )
    
    
    [
        lambda df:
        (df['bidPrice'] > 0)
        & (df['askPrice'] > 0)
        & (df['last'] > 0)
    ]
    
    
    .assign(
            spd_pct = lambda df: (df['askPrice'] / df['bidPrice'] - 1) * 100,
            pct_chg = lambda df: (df['last'] / df['prev_close'] - 1) * 100,
        )
    
        
)
    
clean_df

,bidPrice,askPrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,
PBA,38.16,38.18,37.86,38.185,0.052411,0.858426
PGC,32.46,32.60,32.70,32.470,0.431300,-0.703364
URA,19.59,23.26,21.62,21.530,18.734048,-0.416281
AMWD,45.80,54.41,50.17,49.840,18.799127,-0.657764
CHKP,115.23,135.64,124.63,124.995,17.712401,0.292867
...,...,...,...,...,...,...
OVID,2.01,2.19,2.06,2.000,8.955224,-2.912621
LGF.B,8.35,9.08,8.30,8.360,8.742515,0.722892
SARK,51.12,55.76,54.76,55.750,9.076682,1.807889


In [20]:
@interact(
    max_spread = widgets.IntSlider(value=len(clean_df), min=0, max=len(clean_df)/2, step=25)
)
def filter_liquidity(max_spread):
    global df
    df = clean_df.copy().sort_values(by='spd_pct').iloc[:max_spread]
    #plt.plot(clean_df.index, clean_df.sort_values(by='spd_pct')['spd_pct'].values, label='Spread %')
    plt.figure(figsize=(12,5))
    
    plt.plot(clean_df.sort_values(by='spd_pct').spd_pct.values, label='Universe')
    plt.plot(df.spd_pct.values, label='Liquid Securities')
    plt.legend()
    
    
    print(
        
    '\n',
    f"Number of securities: {len(df)}",
    f"Max Spread: {df.spd_pct.max()}",
    f"Top Gainer = {df[df.pct_chg == df.pct_chg.max()].index[0]}: {df.pct_chg.max()}%",
    f"Top Loser = {df[df.pct_chg == df.pct_chg.min()].index[0]}: {df.pct_chg.min()}%",
        '\n\n',
        sep=' | '
    
    )
    
    
    
    plt.show()


    
    print('Top Gainers')
    display(
        df.sort_values(by='pct_chg', ascending=False).head(15)
    )
    
    print('Top Losers')
    display(
        df.sort_values(by='pct_chg', ascending=True).head(15)
    )
    
    
    
    #return df
    


interactive(children=(IntSlider(value=3360, description='max_spread', max=3360, step=25), Output()), _dom_clas…

In [21]:
df

,bidPrice,askPrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,
SHV,110.10,110.11,110.105,110.105,0.009083,0.000000
SGOV,100.13,100.14,100.125,100.130,0.009987,0.004994
BIL,91.47,91.48,91.470,91.465,0.010933,-0.005466
SHY,83.06,83.07,83.035,83.055,0.012039,0.024086
AAPL,162.17,162.19,162.430,162.200,0.012333,-0.141599
...,...,...,...,...,...,...
EGIO,2.52,2.53,2.530,2.530,0.396825,0.000000
GCO,55.31,55.53,56.080,55.340,0.397758,-1.319544
ATEC,7.54,7.57,7.540,7.570,0.397878,0.397878


In [23]:
print('Top Gainers')
df.sort_values(by='pct_chg', ascending=False).head(15)

Top Gainers


,bidPrice,askPrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,
UGA,64.44,64.56,62.260,64.820,0.186220,4.111789
TMF,13.80,13.82,13.300,13.770,0.144928,3.533835
CANO,6.31,6.33,6.110,6.315,0.316957,3.355155
NUVB,2.84,2.85,2.755,2.845,0.352113,3.266788
PAX,14.36,14.40,13.970,14.400,0.278552,3.078024
NWL,20.77,20.80,20.220,20.780,0.144439,2.769535
SQQQ,40.90,40.92,39.820,40.890,0.048900,2.687092
CIB,29.79,29.84,29.015,29.790,0.167842,2.671032
ZTO,26.26,26.28,25.595,26.270,0.076161,2.637234


In [24]:
print('Top Losers')
df.sort_values(by='pct_chg', ascending=True).head(15)

Top Losers


,bidPrice,askPrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,
BAK,13.28,13.32,14.240,13.290,0.301205,-6.671348
STEM,10.75,10.77,11.320,10.765,0.186047,-4.902827
SNBR,42.86,42.94,45.060,42.900,0.186654,-4.793609
VRAY,2.92,2.93,3.065,2.920,0.342466,-4.730832
INFN,6.25,6.27,6.565,6.270,0.320000,-4.493526
MGNX,2.97,2.98,3.125,2.985,0.336700,-4.480000
QS,10.32,10.34,10.820,10.340,0.193798,-4.436229
HBM,3.69,3.70,3.860,3.690,0.271003,-4.404145
ARRY,16.10,16.12,16.850,16.120,0.124224,-4.332344
